In [3]:
from google.colab import drive
drive.mount('/content/drive/')
%cd drive/MyDrive/Colab Notebooks/Thesis Project/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/Thesis Project/'
/content/drive/MyDrive/Colab Notebooks/Thesis Project


In [4]:
!ls

'0. All Datasets_Processed .gsheet'
'0. All Datasets_Processed.xlsx'
'0. All Datasets.xlsx'
'0. Depression Analysis Dataset.xlsx'
'1. Depression Analysis Dataset Cleaned v1.xlsx'
'1. Depression Analysis Dataset Cleaned v2.xlsx'
'1. Depression Analysis Dataset Cleaned v3_textcleaned.xlsx'
'1. Depression Analysis Dataset Cleaned v3_textstemmed.xlsx'
'1. Depression Analysis v1.ipynb'
'2.1 Depression Analysis v2 (Previous code).ipynb'
'2.2 Depression Analysis v2.ipynb'
'2.3 Depression Analysis v2 modified.ipynb'
'3.1 Depression_Analysis_v3.ipynb'
'3.2 Depression_Analysis_v3 (Word2Vec).ipynb'
'3.3 Depression_Analysis_v3 (FastText).ipynb'
'3.4 Depression_Analysis_LSTM_FastText.ipynb'
'3.4 Depression_Analysis_v3 (Glove).ipynb'
'3.5 Depression_Analysis_BiLSTM_FastText.ipynb'
'4.1 Depression_Analysis_v4 (New Tokenizer).ipynb'
'4.2 Depression_Analysis_v4_Lemmatize.ipynb'
'4.3 Depression_Analysis_v4_Stem.ipynb'
'4.4 Depression_Analysis (Hyperparameter Tuning).ipynb'
'5.1 BanglaBert.ipynb'
'5.2 Be

In [5]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.6 MB/s eta 0:00:00


In [6]:
# pip install git+https://github.com/csebuetnlp/normalizer #BanglaBert

In [7]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from transformers import BertModel, BertTokenizer
from transformers import AutoTokenizer
# from transformers import AutoModelForPreTraining, ElectraModel, ElectraTokenizer #BanglaBert
from transformers import AlbertModel, PreTrainedTokenizerFast #sahajBERT
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [8]:
print(torch.__version__)

2.0.1+cu118


In [13]:
# Load the data
df = pd.read_excel('0. All Datasets.xlsx')
# df = pd.read_excel('0. All Datasets_Processed.xlsx')
df['label'] = df['label'].map({'Non Depressive': 0, 'Depressive': 1})
# Drop rows with any missing values
df = df.dropna(subset=['text', 'label'])
# drop rows that have identical values in all columns
df.drop_duplicates(inplace=True)

# Assuming your DataFrame is named 'df'
columns_to_drop = [col for col in df.columns if col not in ['text', 'label']]
df.drop(columns=columns_to_drop, inplace=True)

print(df.info())

label_counts = df['label'].value_counts()
print("Label Details:")
print(label_counts)

df.sample(n=10)  # Change the value of 'n' to the desired number of random samples

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31695 entries, 0 to 31698
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    31695 non-null  object
 1   label   31695 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 742.9+ KB
None
Label Details:
0    16719
1    14976
Name: label, dtype: int64


,text,label
23558,আমি যতটা বড় মহাকাব্যিক ছবিগুলি পছন্দ করি - আম...,0
16722,স্কুল অর্থহীন যদি আপনি এমন কিছুর জন্য অধ্যয়ন ...,0
1724,"তাই কিছু পিছনের গল্প, আমার বোন সবসময় আমার সের...",1
6611,সত্যিই ঘৃণা করি এই মহামারী চলাকালীন আমার মস্তি...,0
22775,আপনি কি আমাকে এই বাচ্চাটির শো খুঁজে পেতে সাহায...,0
3639,"আমার গ্রেড কমে যাচ্ছে এবং আমি আতঙ্কিত, কিন্তু ...",0
13383,"হ্যালো, আমার বাবা যিনি বেশ আপত্তিজনক এবং আমার ...",1
7752,"মাফ করবেন আমার ব্যাকরণ, ইংরেজি আমার প্রথম ভাষা...",1
16581,একটি টয়লেট যা একটি ড্রেসারের মতো ডিজাইন করা হ...,0
5349,আমি আপনি গ্রাহ্য না জানি. এখন কয়েক মাস হয়ে গ...,1


In [14]:
# Sample 14,000 instances for each label
df_label_0 = df[df['label'] == 0].sample(n=14000)
df_label_1 = df[df['label'] == 1].sample(n=14000)

# Concatenate the sampled dataframes
df = pd.concat([df_label_0, df_label_1], ignore_index=True)

print(df.info())

label_counts = df['label'].value_counts()
print("Label Details:")
print(label_counts)

df.sample(n=10)  # Change the value of 'n' to the desired number of random samples

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    28000 non-null  object
 1   label   28000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 437.6+ KB
None
Label Details:
0    14000
1    14000
Name: label, dtype: int64


,text,label
834,জিনো কস্তা (ম্যাসিমো গিরোত্তি) একজন তরুণ এবং স...,0
24101,সাহায্য করুন. কয়েক বছর ধরে আমি একটি গুরুতর বি...,1
7910,অনুগ্রহ করে সাহায্য করুন আমার একটি জ্ঞানীয় অস...,0
5794,সবেমাত্র গতকাল মনস্টার ম্যানকে দেখতে পেয়েছিলা...,0
4563,বেশিরভাগ মেয়েই জানে না তারা আসলে কি চায়। একট...,0
22899,আমি এবং আমার মা যুগ যুগ ধরে একত্রিত হইনি। আমি ...,1
6664,আমি কি করব?!?! তাই প্রায় 2 বছর আগে আমি 8ম শ্র...,0
5790,আমার কি আমার ভাইয়ের মুখোমুখি হতে হবে? আমার ভা...,0
5769,আমি এটা ঘৃণা করি যখন লোকেরা এমন আচরণ করে যে তা...,0
7356,দয়া করে লোকেদের শ্বাসরোধ করবেন না তাই আমি একট...,0


In [15]:
X = df['text']
y = df['label']
print(X.info())
print(y.info())

<class 'pandas.core.series.Series'>
RangeIndex: 28000 entries, 0 to 27999
Series name: text
Non-Null Count  Dtype 
--------------  ----- 
28000 non-null  object
dtypes: object(1)
memory usage: 218.9+ KB
None
<class 'pandas.core.series.Series'>
RangeIndex: 28000 entries, 0 to 27999
Series name: label
Non-Null Count  Dtype
--------------  -----
28000 non-null  int64
dtypes: int64(1)
memory usage: 218.9 KB
None


In [17]:
# Define batch size
BATCH_SIZE = 32
LEARNING_RATE = 0.01

DROPOUT = 0.1

# Training loop
NUM_EPOCHS = 10

# Define number of folds for cross-validation
NUM_FOLDS = 5

# Momentum -> hyperparameter of Stochastic Gradient Descent (SGD) optimizer
MOMENTUM = 0.9


# Define the BERT model architecture

In [18]:
# Define the BERT model architecture
class BERTClassifier(nn.Module):
    def __init__(self, bert_model, hidden_dim, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(DROPOUT)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = output['pooler_output']
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        return torch.sigmoid(logits)


# Set device
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# model_identifier = 'bert-base-multilingual-cased'
# model_identifier ='csebuetnlp/banglabert'
model_identifier = 'neuropark/sahajBERT'

# Load the BERT tokenizer and encode the text data
# tokenizer = BertTokenizer.from_pretrained(model_identifier)
# tokenizer = AutoTokenizer.from_pretrained('csebuetnlp/banglabert')
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_identifier)

X_encodings = tokenizer(list(X), truncation=True, padding=True, max_length=512)

# Convert labels to tensor
y_tensor = torch.tensor(y.values)

# Initialize the StratifiedKFold
skf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X_encodings['input_ids'], y_tensor), 1):
    print(f"Processing Fold: {fold}")

    # Convert X_encodings to a list of tuples
    X_encodings_list = list(X_encodings.items())

    # Split data into train and validation sets
    X_train_fold = {key: [value[i] for i in train_index] for key, value in X_encodings_list}
    X_val_fold = {key: [value[i] for i in val_index] for key, value in X_encodings_list}
    y_train_fold = y_tensor[train_index]
    y_val_fold = y_tensor[val_index]


    # Create PyTorch datasets
    train_dataset = TensorDataset(torch.tensor(X_train_fold['input_ids']),
                                  torch.tensor(X_train_fold['attention_mask']),
                                  torch.tensor(y_train_fold))
    val_dataset = TensorDataset(torch.tensor(X_val_fold['input_ids']),
                                torch.tensor(X_val_fold['attention_mask']),
                                torch.tensor(y_val_fold))
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

Processing Fold: 1


<ipython-input-18-774b713c160b>:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y_train_fold))
<ipython-input-18-774b713c160b>:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y_val_fold))


Processing Fold: 2
Processing Fold: 3
Processing Fold: 4
Processing Fold: 5


In [19]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define the BERT classifier model
class BERTClassifier(nn.Module):
    def __init__(self, bert_model, hidden_dim, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert_model = bert_model
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, input_ids, attention_mask):
        output = self.bert_model(input_ids, attention_mask)[1]  # Use the [CLS] token representation
        logits = self.fc(output)
        return logits

# Set the device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the BERT model
# bert_model = BertModel.from_pretrained(model_identifier)
bert_model = AlbertModel.from_pretrained("neuropark/sahajBERT")


hidden_dim = bert_model.config.hidden_size
num_classes = 2
#bert_model = torch.compile(bert_model)
model = BERTClassifier(bert_model, hidden_dim, num_classes).to(DEVICE)
# model = torch.compile(model)

# Define the optimizer and loss function
# optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

criterion = nn.CrossEntropyLoss()

best_accuracy = 0.0

# Freeze BERT layer
for param in model.bert_model.parameters():
    param.requires_grad = False

# Unfreeze output layer
for param in model.fc.parameters():
    param.requires_grad = True

# Define lists to store metrics for each fold
fold_train_losses = []
fold_train_accuracies = []
fold_val_losses = []
fold_val_accuracies = []
fold_val_predictions = []
fold_val_labels = []

# Define lists to store train and validation metrics per epoch
train_losses_per_epoch = []
train_accuracies_per_epoch = []
val_losses_per_epoch = []
val_accuracies_per_epoch = []

for epoch in range(NUM_EPOCHS):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for batch in train_loader:
      input_ids, attention_mask, labels = batch
      input_ids = input_ids.to(DEVICE)
      attention_mask = attention_mask.to(DEVICE)
      labels = labels.to(DEVICE)

      optimizer.zero_grad()

      logits = model(input_ids, attention_mask)
      _, predicted = torch.max(logits, 1)

      loss = criterion(logits, labels)
      loss.backward()
      optimizer.step()

      train_loss += loss.item() * input_ids.size(0)
      train_total += labels.size(0)
      train_correct += (predicted == labels).sum().item()

    train_accuracy = 100 * train_correct / train_total

    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for batch in val_loader:
          input_ids, attention_mask, labels = batch
          input_ids = input_ids.to(DEVICE)
          attention_mask = attention_mask.to(DEVICE)
          labels = labels.to(DEVICE)

          logits = model(input_ids, attention_mask)
          _, predicted = torch.max(logits, 1)

          loss = criterion(logits, labels)

          val_loss += loss.item() * input_ids.size(0)
          val_total += labels.size(0)
          val_correct += (predicted == labels).sum().item()

          fold_val_predictions.extend(predicted.cpu().numpy())
          fold_val_labels.extend(labels.cpu().numpy())

    val_accuracy = 100 * val_correct / val_total

    fold_train_losses.append(train_loss / train_total)
    fold_train_accuracies.append(train_accuracy)
    fold_val_losses.append(val_loss / val_total)
    fold_val_accuracies.append(val_accuracy)

    train_losses_per_epoch.append(train_loss / train_total)
    train_accuracies_per_epoch.append(train_accuracy)
    val_losses_per_epoch.append(val_loss / val_total)
    val_accuracies_per_epoch.append(val_accuracy)

    print('Epoch: {}/{} | Train Loss: {:.4f} | Train Accuracy: {:.2f}% | Val Loss: {:.4f} | Val Accuracy: {:.2f}%'.format(
        epoch + 1, NUM_EPOCHS, train_loss / train_total, train_accuracy, val_loss / val_total, val_accuracy))

    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        torch.save(model.state_dict(), 'best_model.pth')

# Calculate performance metrics
average_accuracy = accuracy_score(fold_val_labels, fold_val_predictions)
average_precision = precision_score(fold_val_labels, fold_val_predictions)
average_recall = recall_score(fold_val_labels, fold_val_predictions)
average_f1_score = f1_score(fold_val_labels, fold_val_predictions)

# Print average metrics across folds
print('Average Metrics Across Folds:')
print('Accuracy: {:.4f} | Precision: {:.4f} | Recall: {:.4f} | F1 Score: {:.4f}'.format(
    average_accuracy, average_precision, average_recall, average_f1_score))

# Plotting the train and validation accuracy and loss graphs
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses_per_epoch, label='Train Loss')
plt.plot(val_losses_per_epoch, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies_per_epoch, label='Train Accuracy')
plt.plot(val_accuracies_per_epoch, label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


Some weights of the model checkpoint at neuropark/sahajBERT were not used when initializing AlbertModel: ['sop_classifier.classifier.bias', 'predictions.dense.bias', 'predictions.bias', 'predictions.decoder.bias', 'predictions.LayerNorm.weight', 'predictions.dense.weight', 'predictions.decoder.weight', 'sop_classifier.classifier.weight', 'predictions.LayerNorm.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


KeyboardInterrupt: ignored

In [ ]:
# Define the BERT model architecture
class BERTClassifier(nn.Module):
    def __init__(self, bert_model, hidden_dim, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(DROPOUT)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = output['pooler_output']
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        return torch.sigmoid(logits)


# Set device
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load the BERT tokenizer and encode the text data
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# tokenizer = AutoTokenizer.from_pretrained('csebuetnlp/banglabert')
tokenizer = PreTrainedTokenizerFast.from_pretrained("neuropark/sahajBERT")

# X_normalized = [normalize(text) for text in X]

X_encodings = tokenizer(list(X), truncation=True, padding=True, max_length=512)
# X_encodings = tokenizer(list(X_normalized), truncation=True, padding=True, max_length=512)


# Convert labels to tensor
y_tensor = torch.tensor(y.values)

# Initialize the StratifiedKFold
skf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X_encodings['input_ids'], y_tensor), 1):
    print(f"Processing Fold: {fold}")

    # Convert X_encodings to a list of tuples
    X_encodings_list = list(X_encodings.items())

    # Split data into train and validation sets
    X_train_fold = {key: [value[i] for i in train_index] for key, value in X_encodings_list}
    X_val_fold = {key: [value[i] for i in val_index] for key, value in X_encodings_list}
    y_train_fold = y_tensor[train_index]
    y_val_fold = y_tensor[val_index]


    # Create PyTorch datasets
    train_dataset = TensorDataset(torch.tensor(X_train_fold['input_ids']),
                                  torch.tensor(X_train_fold['attention_mask']),
                                  torch.tensor(y_train_fold))
    val_dataset = TensorDataset(torch.tensor(X_val_fold['input_ids']),
                                torch.tensor(X_val_fold['attention_mask']),
                                torch.tensor(y_val_fold))

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)


Processing Fold: 1


/tmp/ipykernel_29/1196739732.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y_train_fold))
/tmp/ipykernel_29/1196739732.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y_val_fold))


Processing Fold: 2
Processing Fold: 3
Processing Fold: 4
Processing Fold: 5


In [ ]:
torch.cuda.empty_cache()


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define the BERT classifier model
class BERTClassifier(nn.Module):
    def __init__(self, bert_model, hidden_dim, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert_model = bert_model
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, input_ids, attention_mask):
        output = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = output.last_hidden_state[:, 0, :]  # Use the [CLS] token representation
        logits = self.fc(cls_output)
        return logits

# Set the device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the BERT model
# bert_model = ElectraModel.from_pretrained('csebuetnlp/banglabert')
bert_model = AlbertModel.from_pretrained("neuropark/sahajBERT")

hidden_dim = bert_model.config.hidden_size
num_classes = 2

model = BERTClassifier(bert_model, hidden_dim, num_classes).to(DEVICE)

# Define the optimizer and loss function
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
criterion = nn.CrossEntropyLoss()

best_accuracy = 0.0

# Freeze BERT layer
for param in model.bert_model.parameters():
    param.requires_grad = False

# Unfreeze output layer
for param in model.fc.parameters():
    param.requires_grad = True

# Define lists to store metrics for each fold
fold_train_losses = []
fold_train_accuracies = []
fold_val_losses = []
fold_val_accuracies = []
fold_val_predictions = []
fold_val_labels = []

# Define lists to store train and validation metrics per epoch
train_losses_per_epoch = []
train_accuracies_per_epoch = []
val_losses_per_epoch = []
val_accuracies_per_epoch = []

for epoch in range(NUM_EPOCHS):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(DEVICE)
        attention_mask = attention_mask.to(DEVICE)
        labels = labels.to(DEVICE)

        optimizer.zero_grad()

        logits = model(input_ids, attention_mask)
        _, predicted = torch.max(logits, 1)

        # Convert labels to one-dimensional tensor
        labels = labels.squeeze()

        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * input_ids.size(0)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_accuracy = 100 * train_correct / train_total

    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(DEVICE)
            attention_mask = attention_mask.to(DEVICE)
            labels = labels.to(DEVICE)

            logits = model(input_ids, attention_mask)
            _, predicted = torch.max(logits, 1)

            loss = criterion(logits, labels)

            val_loss += loss.item() * input_ids.size(0)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

            fold_val_predictions.extend(predicted.cpu().numpy())
            fold_val_labels.extend(labels.cpu().numpy())

    val_accuracy = 100 * val_correct / val_total

    fold_train_losses.append(train_loss / train_total)
    fold_train_accuracies.append(train_accuracy)
    fold_val_losses.append(val_loss / val_total)
    fold_val_accuracies.append(val_accuracy)

    train_losses_per_epoch.append(train_loss / train_total)
    train_accuracies_per_epoch.append(train_accuracy)
    val_losses_per_epoch.append(val_loss / val_total)
    val_accuracies_per_epoch.append(val_accuracy)

    print('Epoch: {}/{} | Train Loss: {:.4f} | Train Accuracy: {:.2f}% | Val Loss: {:.4f} | Val Accuracy: {:.2f}%'.format(
        epoch + 1, NUM_EPOCHS, train_loss / train_total, train_accuracy, val_loss / val_total, val_accuracy))

    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        torch.save(model.state_dict(), 'best_model.pth')

# Calculate performance metrics
average_accuracy = accuracy_score(fold_val_labels, fold_val_predictions)
average_precision = precision_score(fold_val_labels, fold_val_predictions)
average_recall = recall_score(fold_val_labels, fold_val_predictions)
average_f1_score = f1_score(fold_val_labels, fold_val_predictions)

# Print average metrics across folds
print('Average Metrics Across Folds:')
print('Accuracy: {:.4f} | Precision: {:.4f} | Recall: {:.4f} | F1 Score: {:.4f}'.format(
    average_accuracy, average_precision, average_recall, average_f1_score))

# Plotting the train and validation accuracy and loss graphs
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses_per_epoch, label='Train Loss')
plt.plot(val_losses_per_epoch, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies_per_epoch, label='Train Accuracy')
plt.plot(val_accuracies_per_epoch, label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Print hyperparameters
print('Hyperparameters:')
print('Batch Size:', BATCH_SIZE)
print('Learning Rate:', LEARNING_RATE)
print('Dropout:', DROPOUT)
print('Number of Epochs:', NUM_EPOCHS)
print('Number of Folds:', NUM_FOLDS)
print('Momentum:', MOMENTUM)

# Print average metrics across folds
print('Average Metrics Across Folds:')
print('Accuracy: {:.4f} | Precision: {:.4f} | Recall: {:.4f} | F1 Score: {:.4f}'.format(
    average_accuracy, average_precision, average_recall, average_f1_score))